#Подготовка: установка требуемых Python-библиотек и зависимостей

In [2]:
!pip install groq
!pip install -U langchain-community
!pip install wikipedia
!pip install faiss-cpu
!pip install tqdm
!pip install keybert


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.7 MB/s eta 0:00:00


In [8]:
import spacy
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.outputs import ChatGeneration, ChatResult
from groq import Groq
from typing import List, Dict, Any, Optional, Iterator
import os
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import WikipediaLoader
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from tqdm import tqdm
import json
import re


#Описание класса GroqLlama3Chat

Данный класс реализует обёртку для взаимодействия с языковой моделью Llama-4-Maverick через внешний API Groq. Он наследуется от базового класса BaseChatModel и обеспечивает отправку сообщений и получение ответов от модели.

Основные функции класса:

Инициализация параметров: Включает API-ключ, выбор модели (meta-llama/llama-4-maverick-17b-128e-instruct), настройки температуры генерации и максимального количества токенов в ответе.

Метод _convert_messages: Преобразует входные сообщения из внутреннего формата в формат, понятный API Groq, определяя роли пользователя, системы и ассистента.

Метод _generate: Отправляет запрос к API Groq с подготовленными сообщениями, получает ответ и возвращает его в виде объекта ChatResult, содержащего сгенерированное сообщение.

Свойство _llm_type: Возвращает строковый идентификатор модели.

Данный класс позволяет интегрировать Llama-4-Maverick в систему с унифицированным интерфейсом общения с языковыми моделями.

Вставте в поле api_key ключ который можно получить на сайте https://console.groq.com/keys

In [9]:
class GroqLlama3Chat(BaseChatModel):
    api_key: str = "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"
    model: str = "meta-llama/llama-4-maverick-17b-128e-instruct"
    temperature: float = 0.7
    max_tokens: int = 1024

    def _convert_messages(self, messages: List[BaseMessage]) -> List[Dict]:
        converted = []
        for msg in messages:
            if isinstance(msg, HumanMessage):
                role = "user"
            elif isinstance(msg, SystemMessage):
                role = "system"
            else:
                role = "assistant"
            converted.append({"role": role, "content": msg.content})
        return converted

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager = None,
        **kwargs
    ) -> ChatResult:
        client = Groq(api_key=self.api_key)

        response = client.chat.completions.create(
            messages=self._convert_messages(messages),
            model=self.model,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            stream=False,
            stop=stop
        )

        message = AIMessage(content=response.choices[0].message.content)
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    @property
    def _llm_type(self) -> str:
        return "groq-llama3"

In [10]:
llm_GroqLlama = GroqLlama3Chat()

#Данный модуль содержит набор шаблонов подсказок (prompt templates), используемых для различных этапов автоматической проверки фактов и обработки текстов, сгенерированных большими языковыми моделями.

Основные функции промтов:

prompt_for_fact_extractor — шаблон для разбиения сложного предложения на атомарные, независимые фактические утверждения с сохранением контекста. Позволяет извлечь до 5 проверяемых фактов из одного предложения, избегая тривиальных и общих истин.

prompt3 — шаблон для верификации исходного утверждения на основе контекста из Википедии. В зависимости от совпадения с контекстом возвращает либо подтверждённое утверждение, либо исправленное, либо сообщение об отсутствии подтверждающих данных.

prompt4 — шаблон для исправления исходного утверждения, строго следуя контексту Википедии. Корректирует только необходимые части, сохраняя структуру и имена без добавления новых фактов.

extract_context_prompt — шаблон для извлечения ключевой информации из биографического текста в структурированном JSON-формате. Выделяет имя, профессию, дату и место рождения и другие уникальные характеристики.

conclusions — шаблон для редактирования текста с учётом списка проверенных фактических подсказок. Исправляет неточности, заполняет пробелы, не добавляя неподтверждённую информацию.

promt_to_indicate_the_fact — шаблон для классификации факта как подтверждённого ("S") или неподтверждённого ("NS") по заданному контексту. Выдаёт строго однозначный ответ без объяснений, основываясь на точном соответствии и отсутствии противоречий.



In [11]:

prompt_for_fact_extractor = ChatPromptTemplate.from_template(
"""
You are an assistant for extracting meaningful atomic facts from a sentence. Your task is to split the sentence into non-trivial, independently verifiable factual statements that retain enough context.

You will be given a short context text containing key information such as full name, profession, nationality, place and date of birth, and other relevant traits.

Instructions:
1. Break down the input into clear, self-contained factual statements max 5.
2. Use the provided context text to add necessary details in each fact, so that each fact can be independently and unambiguously verified.
3. Avoid overly obvious facts that are universally true and not useful for fact-checking (e.g., "This person is a human", "1949 is a year").
4. Each fact must be verifiable on its own and preserve relevant context.
5. Include non-obvious implications if they help preserve key information.
6. Output the facts as complete sentences.
7. Separate each fact with (*) only — do not use bullets, numbers, or any other text.
8. Do not repeat facts or include redundant rephrasings.

Example Input:
Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida.

Example Context:
Full name: Lanny Flaherty.

Example Output:
Lanny Flaherty is an American actor (*) Lanny Flaherty was born on December 18, 1949 (*) Lanny Flaherty was born in Pensacola, Florida (*) Pensacola is in Florida

---

Input sentence:
{input}

Context:
Full name: {context}
"""
)

prompt3 = ChatPromptTemplate.from_template('''

You are a fact-checking assistant. Your task is to verify the Original Statement using the provided Wikipedia Context. Follow these rules strictly:

1. If the Context DOES NOT CONTAIN INFORMATION about the statement:
   - Return exactly: "No supporting evidence found in the provided context."

2. If the Context CONFIRMS the statement is correct:
   - Return the Original Statement exactly as is.

3. If the Context CONTRADICTS the statement:
   - Return the CORRECTED version of the statement using the information from the Context.

4. Do not include explanations, reasoning, or additional commentary. Only return the final output as specified.

Original Statement: {input}

Wikipedia Context: {context}

Output:''')

prompt4 = ChatPromptTemplate.from_template('''

You are a fact-verification assistant. Your task is to modify the Original Statement according to the Wikipedia Context following these exact rules:

 Output Requirements:
   - ALWAYS maintain the original sentence structure when possible
   - ONLY modify the parts that require correction
   - NEVER add new information not present in either statement or context
   - PRESERVE proper nouns, names, and technical terms exactly as written
   - OUTPUT ONLY the final verified statement with no additional text

Original Statement: {input}

Wikipedia Context: {context}

Verified Statement:''')
extract_context_prompt = ChatPromptTemplate.from_template("""
You are an assistant specialized in analyzing biographical texts.

Carefully extract and clearly identify the following key contextual information from the given text:
- Full name of the individual
- Entity type (e.g., person, organization)
- Profession or occupation, if specified
- Nationality or citizenship
- Place of birth and date of birth
- Any other unique and relevant identifiers or distinguishing characteristics

Provide the output strictly in valid JSON format, with clearly labeled fields corresponding to the above categories.

Text:
{text}

Respond only with the JSON object, without additional commentary or explanation.
""")
conclusions = ChatPromptTemplate.from_template("""
You are an expert assistant in fact-checking and text refinement.

Below is a list of key factual hints (context) extracted from verified information:

{hints}

Now, you will be given a text that may contain inaccuracies, ambiguities, or missing information.

Your task is to revise this text so that it fully aligns with the factual hints provided, correcting any mistakes, filling gaps, and making the text clear and factually accurate.

Do not add any information not supported by the hints.

Text to correct:
{text_to_fix}

Revised text:

"""
)

promt_to_indicate_the_fact = ChatPromptTemplate.from_template('''
Analyze the following statement and context STRICTLY. Return ONLY "S" or "NS" without any explanations:

1. Return "S" ONLY IF:
   - The context EXPLICITLY STATES the fact
   - The fact is mentioned VERBATIM or with SYNONYMS that preserve original meaning
   - There are NO CONTRADICTIONS

2. Return "NS" in ALL OTHER CASES including:
   - No information found
   - Partial matches
   - Contradictions
   - Ambiguous phrasing
   - Implied but not stated facts

Original Statement: {input}
Wikipedia Context: {context}

Output:

''')


В данном блоке реализуются основные компоненты системы для извлечения фактов, контекста и верификации утверждений с помощью большой языковой модели GroqLlama и заранее подготовленных промтов.

sent_chain — цепочка для извлечения атомарных фактов из входного предложения. Использует шаблон prompt_for_fact_extractor, который разбивает сложные предложения на отдельные проверяемые утверждения.

extract_context_chain — цепочка для выделения ключевой информации из биографического текста. Применяет шаблон extract_context_prompt для извлечения структурированного контекста в формате JSON.

check_fackt — цепочка для первичной проверки фактов. Использует шаблон prompt3, который сравнивает исходное утверждение с контекстом из Википедии и возвращает либо подтверждённое, либо исправленное утверждение, либо сообщение об отсутствии доказательств.

fact_checker — цепочка для точечной корректировки утверждений. Применяет шаблон prompt4, который сохраняет структуру исходного предложения, корректируя только выявленные ошибки на основе верифицированного контекста.

fin_conclusions — цепочка для окончательной обработки и генерации исправленного текста. Использует шаблон conclusions для объединения всех исправлений и уточнений в единый связный текст.

indicate_the_fact — цепочка для классификации утверждений как подтверждённых ("S") или неподтверждённых ("NS") на основании анализа контекста. Использует шаблон promt_to_indicate_the_fact, который даёт однозначный ответ без дополнительного текста.

In [12]:
sent_chain= LLMChain(llm=llm_GroqLlama,prompt=prompt_for_fact_extractor)

extract_context_chain = LLMChain(
    llm=llm_GroqLlama,
    prompt=extract_context_prompt
)

check_fackt=create_stuff_documents_chain(llm=llm_GroqLlama,prompt=prompt3)
fact_checker=create_stuff_documents_chain(llm=llm_GroqLlama, prompt=prompt4)
fin_conclusions =  LLMChain(llm=llm_GroqLlama,prompt=conclusions)
indicate_the_fact=create_stuff_documents_chain(llm=llm_GroqLlama,prompt=promt_to_indicate_the_fact)

<ipython-input-12-dad26517d6e9>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sent_chain= LLMChain(llm=llm_GroqLlama,prompt=prompt_for_fact_extractor)


#Данный блок реализует обработку входного текста, включая разбиение на предложения, замену местоимений на имена из контекста, а также извлечение структурированной информации.

Инициализация: загружается модель spaCy en_core_web_sm для обработки естественного языка.

Функция extract_json(text):

Извлекает JSON-объект из текста, находя первый и последний символы фигурных скобок.

При отсутствии фигурных скобок вызывает ошибку.

Функция splitting_the_text(text):

Принимает входной текст и возвращает два объекта: список предложений и извлечённый контекст в виде словаря.

Сначала обрабатывает текст через spaCy, разбивая его на предложения.

Для каждого предложения ищет именованные сущности типа PERSON и ORG и сохраняет их в словарь контекста.

Если в контексте найден человек (PERSON), заменяет в предложениях местоимения he, she на имя этого человека для повышения ясности.

Вызывает цепочку extract_context_chain для извлечения контекста из текста в формате JSON.

Преобразует JSON-строку в словарь.

Возвращает список предложений длиной не менее 20 символов и контекст.

Особенности:

Местоимения заменяются только при наличии в контексте сущности типа PERSON, что помогает сделать предложения более самостоятельными для последующей обработки.

Извлечение контекста позволяет получить структурированные данные о ключевых характеристиках из текста.
Функция fact_extractor(sentence: str, context) принимает предложение и контекст и возвращает список атомарных фактов, извлечённых из предложения.

Использует цепочку sent_chain с заранее подготовленным промтом для разбиения предложения на независимые факты.

Разделяет строку фактов по разделителю (*).

Возвращает список чистых фактов (без лишних пробелов).



In [26]:


nlp = spacy.load("en_core_web_sm")
def extract_json(text):
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1:
        raise ValueError("JSON braces not found in text")
    json_str = text[start:end+1]
    return json.loads(json_str)

def splitting_the_text(text):
    if not text:
        return []

    doc = nlp(text)
    sentences = []
    context = {}

    for sent in doc.sents:
        sent_text = sent.text.strip()

        for ent in sent.ents:
            if ent.label_ in ('PERSON', 'ORG'):
                context[ent.text] = ent.label_

        if 'PERSON' in context.values():
            person = next(k for k,v in context.items() if v == 'PERSON')
            sent_text = sent_text.replace(' he ', f' {person} ')
            sent_text = sent_text.replace('He ', f'{person} ')
            sent_text = sent_text.replace(' she ', f' {person} ')
            sent_text = sent_text.replace('She ', f'{person} ')

        sentences.append(sent_text)
    context_json_str = extract_context_chain.run(text=text)
    print(context_json_str)
    data = extract_json(context_json_str)
    print(data)

    return [s for s in sentences if len(s) >= 20], data

def fact_extractor(sentence: str, context):
    facts = sent_chain.run(input=sentence, context=context)
    facts_list = [fact.strip() for fact in facts.split('(*)')]
    return facts_list

 Формирует итоговую исправленную версию текста на основе фактов (hints) и исходного текста.

    Параметры:
    - doc (str): Строка с ключевыми фактами (hints), извлечёнными из проверенных источников.
    - text (str): Исходный текст, содержащий возможные ошибки или неточности.

    Возвращает:
    dict: Словарь с двумя ключами:
        - 'model': исправленный и уточнённый текст, сформированный моделью на основе фактов.
        - 'orig': исходный текст, переданный для проверки.

In [14]:

def resultt(doc: str, text: str) -> dict:
    corrected_text = fin_conclusions.run(hints=doc, text_to_fix=text)
    return {
        'model': corrected_text,
        'orig': text
    }

Класс для извлечения ключевых сущностей и ключевых слов из фактов,
загрузки и обработки статей Википедии в реальном времени, а также
построения векторного индекса и поиска релевантной информации для
фактической проверки.

    Атрибуты:
    - embedding: Объект для создания эмбеддингов текста с использованием модели HuggingFace.
    - model_response: Модель или цепочка для обработки и оценки релевантности найденных данных.
    - text_splitter: Компонент для разбиения текстов на чанки.
    - article_cache: Кэш загруженных статей Википедии для ускорения повторных запросов.
    - nlp: spaCy модель для обработки текста и извлечения сущностей.
    - kb_model: Модель KeyBERT для извлечения ключевых слов.

    Методы:
    - extract_main_entities_and_keywords(fact, top_n_keywords=5) -> List[str]
        Извлекает важные сущности (например, персоны, организации) и ключевые слова из входного факта.

    - clean_title(title: str) -> str
        Очищает строку от нежелательных символов и формирует корректный заголовок.

    - extract_wiki_titles_spacy(fact: str) -> List[str]
        Извлекает заголовки статей Википедии на основе NER (Named Entity Recognition) с помощью spaCy.

    - extract_titles_fallback(text: str) -> List[str]
        Резервный метод для извлечения заголовков из кавычек в тексте.

    - extract_titles_keybert(text: str, top_n: int = 3) -> List[str]
        Извлекает ключевые слова с помощью KeyBERT как запасной вариант.

    - extract_wiki_titles(fact: str) -> List[str]
        Объединяет различные методы извлечения заголовков для максимального охвата.

    - _load_wikipedia_article(page_title: str) -> Optional[Document]
        Загружает и кэширует содержимое статьи Википедии по названию страницы.

    - _create_vector_index(documents: List[Document]) -> Optional[FAISS]
        Разбивает статьи на чанки и создаёт векторный индекс FAISS для поиска.

    - retrieve_hints(fact: str, max_k: int = 10) -> List[str]
        Выполняет поиск релевантных подсказок (фактов) в Википедии для данного факта,
        используя векторный поиск с постепенным увеличением параметра k.

    - batch_retrieve_hints(facts: List[str], top_k: int = 3) -> Dict[str, List[str]]
        Пакетная обработка списка фактов для получения подсказок с использованием retrieve_hints.



  Класс предназначен для интеграции в системы верификации фактов, использующие внешний
  контекст Википедии для проверки утверждений.

In [105]:
import requests
from bs4 import BeautifulSoup
from keybert import KeyBERT

class TTWikipediaRealTimeRetriever:
    def __init__(self,model_response=check_fackt):
        self.embedding = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': False}
        )
        self.model_response=model_response
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        self.article_cache = {}
        self.nlp = spacy.load("en_core_web_sm")
        # self.nlp = spacy.load("en_core_web_trf")
        self.kb_model = KeyBERT("all-MiniLM-L6-v2")
    def extract_main_entities_and_keywords(self, fact: str, top_n_keywords=5) -> List[str]:
        doc = self.nlp(fact)
        # 1. NER сущности
        ner_entities = [ent.text.strip() for ent in doc.ents if ent.label_ in ("PERSON", "ORG", "GPE", "LOC", "WORK_OF_ART", "EVENT")]

        # Уникальность с сохранением порядка
        seen = set()
        ner_entities_unique = []
        for ent in ner_entities:
            clean_ent = self.clean_title(ent)
            if clean_ent.lower() not in seen and len(clean_ent) > 2:
                ner_entities_unique.append(clean_ent)
                seen.add(clean_ent.lower())

        # 2. Ключевые слова KeyBERT
        keywords = self.kb_model.extract_keywords(fact, top_n=top_n_keywords, stop_words='english')
        keywords_list = []
        for kw, _ in keywords:
            clean_kw = self.clean_title(kw)
            if clean_kw.lower() not in seen and len(clean_kw) > 2:
                keywords_list.append(clean_kw)
                seen.add(clean_kw.lower())

        # 3. Объединяем, с приоритетом NER-сущностей
        combined = ner_entities_unique + keywords_list

        return combined


    def clean_title(self, title: str) -> str:
        title = re.sub(r"'s\b", "", title)
        title = re.sub(r"[\"'“”.,!?;:()\[\]{}]", "", title)
        return title.strip()

    def extract_wiki_titles_spacy(self, fact: str) -> List[str]:
        doc = self.nlp(fact)
        raw_titles = [
            ent.text.strip()
            for ent in doc.ents
            if ent.label_ in ("PERSON", "GPE", "ORG", "LOC", "WORK_OF_ART", "PRODUCT", "EVENT")
        ]
        return list(set(self.clean_title(t) for t in raw_titles if len(t) > 2))

    def extract_titles_fallback(self, text: str) -> List[str]:
        titles = re.findall(r'"([^"]+)"', text)
        if titles:
            return [self.clean_title(t) for t in titles]
        return []

    def extract_titles_keybert(self, text: str, top_n: int = 3) -> List[str]:
        keywords = self.kb_model.extract_keywords(text, top_n=top_n, stop_words='english')
        return [self.clean_title(k[0]) for k in keywords]

    def extract_wiki_titles(self, fact: str) -> List[str]:
        titles = self.extract_wiki_titles_spacy(fact)
        if not titles:
            print("[INFO] spaCy ничего не нашёл — fallback на кавычки и KeyBERT.")
            titles = self.extract_titles_fallback(fact)
        if not titles:
            titles = self.extract_titles_keybert(fact)
        return list(set(titles))

    def _load_wikipedia_article(self, page_title: str) -> Optional[Document]:
        if page_title in self.article_cache:
            return self.article_cache[page_title]

        response = requests.get(
            "https://en.wikipedia.org/w/api.php",
            params={
                "action": "parse",
                "format": "json",
                "page": page_title,
                "prop": "text",
                "disabletoc": 1,
            }
        )

        try:
            html = response.json().get("parse", {}).get("text", {}).get("*", "")
            if not html:
                print(f"[!] Статья '{page_title}' не найдена.")
                return {}

            soup = BeautifulSoup(html, "html.parser")
            text = soup.get_text()
            doc = Document(page_content=text, metadata={"source": page_title})
            self.article_cache[page_title] = doc
            return doc

        except Exception as e:
            print(f"[!] Ошибка при загрузке статьи '{page_title}': {e}")
            return {}

    def _create_vector_index(self, documents: List[Document]) -> Optional[FAISS]:
        split_docs = self.text_splitter.split_documents(documents)
        if not split_docs:
            print("[!] Пустой список чанков для индексации.")
            return {}
        return FAISS.from_documents(split_docs, self.embedding)

    def retrieve_hints(self, fact: str, max_k: int = 10) -> List[str]:

        titles = self.extract_main_entities_and_keywords(fact, top_n_keywords=5)

        print(f"[DEBUG] Извлечённые заголовки: {titles}")

        docs = []
        for title in titles[:1]:
            doc = self._load_wikipedia_article(title)
            if doc:
                docs.append(doc)

        if not docs:
            print("[!] Не найдено ни одной статьи.")
            return {}

        vector_store = self._create_vector_index(docs)
        if vector_store is None:
            return {}

        result = ''
        for k in range(2, max_k + 1, 2):
            retriever = vector_store.as_retriever(search_kwargs={"k": k})
            retrieval_chain = create_retrieval_chain(retriever, self.model_response)
            result = retrieval_chain.invoke({'input': fact})

            if result.get('answer') != 'No supporting evidence found in the provided context.':
                print(f"[INFO] Подходящее значение top_k: {k}")
                return result
            # if result.get('answer') != 'NS':
            #     print(f"[INFO] Подходящее значение top_k: {k}")
            #     return result

        return result

    def batch_retrieve_hints(self, facts: List[str], top_k: int = 3) -> Dict[str, List[str]]:
        results = {}
        for fact in tqdm(facts, desc="Обработка фактов"):
            results[fact] = self.retrieve_hints(fact, top_k)
        return results


def verify_fact(fact: str, retriever=ttret):
    hints = ttret.retrieve_hints(fact)
    return hints


#Тестовая демонстрация этапов верефикации текста

In [78]:
l,js=splitting_the_text('Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida. He has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s. Some of his notable film credits include \"King of New York,\" \"The Abyss,\" \"Natural Born Killers,\" \"The Game,\" and \"The Straight Story.\" On television, he has appeared in shows such as \"Law & Order,\" \"The Sopranos,\" \"Boardwalk Empire,\" and \"The Leftovers.\" Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival. He is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.')


```json
{
  "fullName": "Lanny Flaherty",
  "entityType": "person",
  "profession": "actor",
  "nationality": "American",
  "placeOfBirth": "Pensacola, Florida",
  "dateOfBirth": "1949-12-18",
  "distinguishingCharacteristics": ["distinctive looks", "deep gravelly voice"]
}
```
{'fullName': 'Lanny Flaherty', 'entityType': 'person', 'profession': 'actor', 'nationality': 'American', 'placeOfBirth': 'Pensacola, Florida', 'dateOfBirth': '1949-12-18', 'distinguishingCharacteristics': ['distinctive looks', 'deep gravelly voice']}


In [79]:
l

['Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida.',
 'Lanny Flaherty has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s.',
 'Some of his notable film credits include "King of New York," "The Abyss," "Natural Born Killers," "The Game," and "The Straight Story."',
 'On television, Lanny Flaherty has appeared in shows such as "Law & Order," "The Sopranos," "Boardwalk Empire," and "The Leftovers."',
 'Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival.',
 'Lanny Flaherty is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.']

In [80]:
f=fact_extractor(l[4],js['fullName'])

In [81]:
f

['Lanny Flaherty has worked extensively in theater',
 'Lanny Flaherty has worked at the Public Theater',
 'Lanny Flaherty has worked at the New York Shakespeare Festival',
 'The New York Shakespeare Festival is associated with theater productions',
 'The Public Theater is associated with theater productions']

In [82]:
ttret=TTWikipediaRealTimeRetriever()

In [83]:
v = verify_fact('Lanny Flaherty was born on December 18, 1949.',retriever=ttret)

[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', '1949', 'born', 'december']
[INFO] Подходящее значение top_k: 2


In [84]:
v

{'input': 'Lanny Flaherty was born on December 18, 1949.',
 'context': [Document(id='d49324b4-6c21-4d08-8a1c-1a34aff1af42', metadata={'source': 'Lanny Flaherty'}, page_content='Lanny FlahertyBorn(1942-07-27)July 27, 1942Pontotoc, Mississippi, U.S.DiedFebruary 18, 2024(2024-02-18) (aged\xa081)New York City, U.S.Alma\xa0materUniversity of Southern MississippiOccupationActorYears\xa0active1984–2020\nLanny Flaherty (July 27, 1942 – February 18, 2024) was an American actor.'),
  Document(id='28d8bc39-b8cd-402f-b501-58230065673b', metadata={'source': 'Lanny Flaherty'}, page_content='^ "Obituary of Lanny Flaherty". Andrett Funeral Home. Retrieved February 24, 2024.\n\n\nExternal links[edit]\nLanny Flaherty at IMDb\nAuthority control databases InternationalVIAFWorldCatNationalGermanyPeopleDeutsche Biographie')],
 'answer': 'Lanny Flaherty was born on July 27, 1942.'}

In [ ]:
def generate_response(text: str,):

    context = []
    fact_chunks, js = splitting_the_text(text)


    for chunk in tqdm(fact_chunks, desc="Обработка сегментов текста"):
        print(f"\n>>> Обрабатываем предложение/сегмент:\n{chunk}\n")
        facts = fact_extractor(chunk, js)

        for fact in facts:
            print(f'Проверяем факт: "{fact}"')
            result = verify_fact(fact)
            answer = result.get('answer')
            print(f'Результат проверки: {answer}')
            print("-" * 50)
            context.append({
                "segment": chunk,
                "fact": fact,
                "result": answer
            })



    return context
c=generate_response('Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida. He has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s. Some of his notable film credits include \"King of New York,\" \"The Abyss,\" \"Natural Born Killers,\" \"The Game,\" and \"The Straight Story.\" On television, he has appeared in shows such as \"Law & Order,\" \"The Sopranos,\" \"Boardwalk Empire,\" and \"The Leftovers.\" Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival. He is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.')





```json
{
  "fullName": "Lanny Flaherty",
  "entityType": "person",
  "profession": "actor",
  "nationality": "American",
  "placeOfBirth": "Pensacola, Florida",
  "dateOfBirth": "December 18, 1949",
  "distinguishingCharacteristics": ["distinctive looks", "deep gravelly voice"]
}
```
{'fullName': 'Lanny Flaherty', 'entityType': 'person', 'profession': 'actor', 'nationality': 'American', 'placeOfBirth': 'Pensacola, Florida', 'dateOfBirth': 'December 18, 1949', 'distinguishingCharacteristics': ['distinctive looks', 'deep gravelly voice']}


Обработка сегментов текста:   0%|          | 0/6 [00:00<?, ?it/s]


>>> Обрабатываем предложение/сегмент:
Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida.

Проверяем факт: "Lanny Flaherty is an American actor"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', 'actor', 'american']
[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty was an American actor
--------------------------------------------------
Проверяем факт: "Lanny Flaherty was born on December 18, 1949"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', '1949', 'born', 'december']
[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty was born on July 27, 1942
--------------------------------------------------
Проверяем факт: "Lanny Flaherty was born in Pensacola, Florida"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'Pensacola', 'Florida', 'flaherty', 'lanny', 'born']
[INFO] Подходящее значение top_k: 2
Результат проверки: No supporting evidence found in the provided contex

Обработка сегментов текста:  17%|█▋        | 1/6 [00:06<00:33,  6.67s/it]

[INFO] Подходящее значение top_k: 2
Результат проверки: Pensacola is in Florida
--------------------------------------------------

>>> Обрабатываем предложение/сегмент:
Lanny Flaherty has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s.

Проверяем факт: "Lanny Flaherty is an American actor"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', 'actor', 'american']
[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty was an American actor
--------------------------------------------------
Проверяем факт: "Lanny Flaherty's career began in the late 1970s"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', '1970s', 'career', 'began']
[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty's career began in 1984
--------------------------------------------------
Проверяем факт: "Lanny Flaherty has appeared in numerous films"
[DEBUG] Извлечённые за

...

Результат проверки: No supporting evidence found in the provided context.
--------------------------------------------------

>>> Обрабатываем предложение/сегмент:
Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival.

Проверяем факт: "Lanny Flaherty has worked extensively in theater"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'theater', 'flaherty', 'lanny', 'extensively', 'worked']
Результат проверки: No supporting evidence found in the provided context.
--------------------------------------------------
Проверяем факт: "Lanny Flaherty has worked in productions at the Public Theater"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'the Public Theater', 'flaherty', 'productions', 'theater', 'lanny', 'public']
Результат проверки: No supporting evidence found in the provided context.
--------------------------------------------------
Проверяем факт: "Lanny Flaherty has worked in productions at the New 

Обработка сегментов текста:  83%|████████▎ | 5/6 [07:46<01:50, 110.48s/it]

[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty was an American actor with a distinctive appearance and a deep gravelly voice
--------------------------------------------------

>>> Обрабатываем предложение/сегмент:
Lanny Flaherty is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.

Проверяем факт: "Lanny Flaherty is an American actor"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', 'actor', 'american']
[INFO] Подходящее значение top_k: 2
Результат проверки: Lanny Flaherty was an American actor
--------------------------------------------------
Проверяем факт: "Lanny Flaherty has distinctive looks"
[DEBUG] Извлечённые заголовки: ['Lanny Flaherty', 'flaherty', 'lanny', 'distinctive', 'looks']
Результат проверки: No supporting evidence found in the provided context.
--------------------------------------------------
Проверяем факт: "Lanny Flaherty has a deep gravelly voi

Обработка сегментов текста: 100%|██████████| 6/6 [09:18<00:00, 93.11s/it] 

[INFO] Подходящее значение top_k: 2
Результат проверки: No supporting evidence found in the provided context for the birth date and place, but the correct information is available: Lanny Flaherty was born on July 27, 1942, in Pontotoc, Mississippi. 

However, to follow the rules: Lanny Flaherty was born on July 27, 1942, in Pontotoc, Mississippi.
--------------------------------------------------


In [ ]:
l=[]
s=0
for i in c:
  if(i['result']=='No supporting evidence found in the provided context.'):
    s+=1
    continue
  else:
    l.append(i['result'])

In [ ]:
l

['Lanny Flaherty was an American actor',
 'Lanny Flaherty was born on July 27, 1942',
 "No supporting evidence found in the provided context is not applicable here as the context contains information about Lanny Flaherty's birthplace. \nLanny Flaherty was born in Pontotoc, Mississippi.",
 'Pensacola is in Florida',
 'Lanny Flaherty was an American actor',
 "Lanny Flaherty's career began in 1984",
 'Lanny Flaherty has appeared in numerous films',
 'Lanny Flaherty has appeared in numerous television shows',
 'Lanny Flaherty has appeared in the film "Natural Born Killers"',
 'Lanny Flaherty was an American actor who has appeared on television',
 'The Public Theater and the New York Shakespeare Festival are organizations associated with theater productions',
 'Lanny Flaherty was an American actor with a distinctive appearance and a deep gravelly voice',
 'Lanny Flaherty was an American actor',
 'Lanny Flaherty is known for being a memorable character actor',
 'No supporting evidence found 

In [ ]:
s

15

In [ ]:
o=result(l,'Lanny Flaherty is an American actor born on December 18, 1949, in Pensacola, Florida. He has appeared in numerous films, television shows, and theater productions throughout his career, which began in the late 1970s. Some of his notable film credits include \"King of New York,\" \"The Abyss,\" \"Natural Born Killers,\" \"The Game,\" and \"The Straight Story.\" On television, he has appeared in shows such as \"Law & Order,\" \"The Sopranos,\" \"Boardwalk Empire,\" and \"The Leftovers.\" Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival. He is known for his distinctive looks and deep gravelly voice, which have made him a memorable character actor in the industry.')

In [ ]:
o

{'model': 'Here is the revised text, corrected to align with the factual hints provided:\n\nLanny Flaherty was an American actor born on July 27, 1942, in Pontotoc, Mississippi. He has appeared in numerous films and television shows throughout his career, which began in 1984. Some of his notable film credits include "Natural Born Killers." Flaherty has also worked extensively in theater, including productions at the Public Theater and the New York Shakespeare Festival. He is known for his distinctive appearance and deep gravelly voice, which have made him a memorable character actor in the industry.\n\nI made the following changes to the original text:\n\n* Corrected the birthdate to July 27, 1942, instead of December 18, 1949.\n* Corrected the birthplace to Pontotoc, Mississippi, instead of Pensacola, Florida (noting that Pensacola is indeed in Florida, but not the correct birthplace).\n* Changed the start of his career to 1984, instead of the late 1970s.\n* Removed "The Abyss," "King

#Основная функция, которая принимает произвольный текст, проверяет содержащиеся в нём факты через Википедию и возвращает исправленный (фактологически выверенный) текст.

Шаги:
1. Разбивает текст на сегменты и извлекает контекст.
2. Из каждого сегмента извлекает атомарные факты.
3. Для каждого факта ищет подтверждение в Википедии.
4. Собирает подтверждённые факты.
5. Формирует исправленный текст на основе проверенных фактов.

In [ ]:
retriver = TTWikipediaRealTimeRetriever()

def verify_and_rewrite_text(text: str):

    context = []
    verified_facts = []

    fact_chunks, context_json = splitting_the_text(text)

    for chunk in tqdm(fact_chunks, desc="Обработка предложений"):
        print(f"\n>>> Предложение:\n{chunk}")
        facts = fact_extractor(chunk, context_json)

        for fact in facts:
            print(f'  Проверка факта: "{fact}"')
            result = verify_fact(fact)
            answer = result.get('answer')
            print(f'  Ответ: {answer}\n' + "-" * 50)

            if answer != "No supporting evidence found in the provided context.":
                verified_facts.append(answer)

            context.append({
                "segment": chunk,
                "fact": fact,
                "result": answer
            })

    if verified_facts:
        corrected_output = resultt(verified_facts, text)
        return {
            "original": text,
            "verified_facts": verified_facts,
            "corrections": corrected_output["model"]
        }
    else:
        return {
            "original": text,
            "verified_facts": [],
            "corrections": "Не удалось подтвердить ни одного факта."
        }
